# Les _support vector machines_

In [ ]:
## Les imports de base
import numpy as np
import matplotlib.pyplot as plt

## Historique des SVM

Les Support Vector Machines (SVM) ont été introduits dans les années 1990 par Vladimir Vapnik et ses collaborateurs, dans le cadre du développement de la [théorie de Vapnik-Chervonenkis](https://fr.wikipedia.org/wiki/Th%C3%A9orie_de_Vapnik-Chervonenkis) (un nom compliqué pour parler d'apprentissage statistique...). Un SVM est un modèle d'apprentissage supervisé destiné à résoudre des problèmes de classification et de régression en trouvant un hyperplan optimal qui sépare les données en classes distinctes avec une marge maximale.

Le concept de base des SVMs repose sur la minimisation de l'erreur de classification tout en maximisant la marge entre les classes. Ce modèle a été largement popularisé grâce à ses performances remarquables (pour l'époque) et à sa capacité à gérer des espaces de haute dimension, faisant des SVM un choix populaire pour une variété de tâches d'apprentissage automatique.

Avec le temps, plusieurs algorithmes ont été développés pour résoudre les problèmes de SVM, y compris l'[algorithme SMO (_Sequential minimal optimization_)](https://cs229.stanford.edu/materials/smo.pdf) et les méthodes basées sur les points intérieurs. Ces développements ont permis d'améliorer l'efficacité et la scalabilité des SVM dans des applications pratiques.

Les SVMs ont été développés à la base pour des problèmes de classification binaire (mais il est possible d'étendre leur fonctionnement aux problèmes multi-classes). Leur objectif est de trouver un hyperplan qui sépare les données en deux classes de manière optimale. L'idée clé est de maximiser la marge, c'est-à-dire la distance entre les échantillons les plus proches des deux classes et l'hyperplan séparateur. Les points les plus proches de l'hyperplan sont appelés **vecteurs supports**.

## Formulation du problème primal avec marge dure

Étant donné un problème de classification binaire $\{(\mathbf{x}_i, y_i)\}_{i=1}^n$ avec $\mathbf{x}_i \in \mathbb{R}^p$ un échantillon (de dimension $p$) et $y_i \in \{-1,+1\}$ sa classe associée.

Si les données sont **linéairement séparables**, alors il existe un hyperplan $\mathcal{H} = (\mathbf{w},b)$ d'équation $\mathbf{w}^T \mathbf{x} + b = 0$ qui sépare parfaitement les données de la classe $+1$ de celles de la classe $-1$ avec $\mathbf{w} \in \mathbb{R}^p$ un vecteur normal à l'hyperplan, et $b \in \mathbb{R}$ le biais (ordonnée à l'origine).

L'existence d'un tel hyperplan :
* se traduit mathématiquement par le fait que $\mathbf{w}^T \mathbf{x}_i + b \geq 0$ si $y_i = +1$ et $\mathbf{w}^T \mathbf{x}_i + b \leq 0$ si $y_i = -1$, ou, dit de manière plus compacte, 

$$ y_i (\mathbf{w}^T \mathbf{x}_i + b) \geq 0 \ \ \forall i = 1,\dots,n $$
* implique en fait qu'il existe une infinité de tels hyperplan $\Rightarrow$ il faut donc un critère supplémentaire pour choisir l'hyperplan optimal.

Le critère supplémentaire de choix de l'hyperplan optimal dans le cas du SVM est la **maximisation** de la marge, autrement dit, la distance entre l'hyperplan et les échantillons les plus proches des deux côtés de l'hyperplan.
On peut alors montrer (cf le cours) que :
* la marge (normalisée) s'écrit $M_\mathcal{H} = \frac{2}{\| \mathbf{w} \|}$
* les contraintes de séparabilité linéaire se réécrivent $ y_i (\mathbf{w}^T \mathbf{x}_i + b) \geq 1 \ \ \forall i = 1,\dots,n $

La maximisation de la marge est ramenée à une minimisation d'un terme quadratique (totalement équivalent), pour aboutir au final à la formulation suivante du problème primal $(P)$ du SVM à marge dure :

$$\begin{align}
\min_{(\mathbf{w},b) \in \mathbb{R}^{p}\times \mathbb{R}} \quad & \frac{1}{2} \| \mathbf{w} \|^2 \qquad (P)\\
\text{tel que} \quad & y_i (\mathbf{w}^T \mathbf{x}_i + b) \geq 1 \ \forall i = 1,\dots,n
\end{align}
$$

La fonction objective étant quadratique et les contraintes étant linéaires, on est bien en présence d'un problème de programmation quadratique (QP).

## Formulation du problème dual

L'application des conditions KKT au problème primal permettent de formuler le problème dual $(D)$ associé au problème primal $(P)$ précédent pour le SVM, à savoir :

$$\begin{align}
\max_{\boldsymbol \alpha \in \mathbb{R}^{n}} \quad & \sum_{i=1}^n \alpha_i - \frac{1}{2} \sum_{i=1}^n \sum_{j=1}^n \alpha_i \alpha_j y_i y_j \mathbf{x}_i^T \mathbf{x}_j \qquad (D) \\
\text{tel que} \quad & \alpha_i \geq 0 \ \ \forall i = 1, \dots, n \\
& \sum_{i=1}^n \alpha_i y_i = 0
\end{align}
$$

où $\boldsymbol \alpha \in \mathbb{R}^n$ est la variable duale constituée des $n$ multiplicateurs de Lagrange associés aux $n$ contraintes d'inégalité du problème primal.

Le problème dual est également un problème QP, de contraintes $\alpha_i \geq 0 \ \ \forall i = 1,\dots,n$ (admissibilité duale) et $\sum_{i=1}^n \alpha_i y_i = 0$ (relation découlant de la stationnarité du Lagrangien).
Il peut être reformulé sous forme vectorielle comme : 

$$\begin{align}
\max_{\boldsymbol \alpha \in \mathbb{R}^{n}} \quad & \mathbf{1}^T \boldsymbol \alpha - \frac{1}{2} \boldsymbol \alpha^T \mathbf{Q} \boldsymbol \alpha \qquad (D)\\
\text{tel que} \quad & \alpha_i \geq 0 \ \ \forall i = 1, \dots, n \\
& \mathbf{y}^T \boldsymbol \alpha = 0
\end{align}
$$

ou
- $ \mathbf{1} \in \mathbb{R}^n $ est un vecteur de 1.
- $ \mathbf{Q} \in \mathbb{R}^{n \times n} $ est la matrice des produits scalaires des échantillons multipliés par le produit de leur classe $ Q_{ij} = y_i y_j \mathbf{x}_i^T \mathbf{x}_j $.
- $ \mathbf{y} \in \mathbb{R}^n $ est le vecteur des labels, avec $ y_i \in \{-1, 1\} $.

## Solution du problème primal

Une fois la solution optimale $ \boldsymbol \alpha^\star $ du problème dual trouvée, on peut en déduire la solution optimale du problème primal de la manière suivante :

- **Vecteur normal à l'hyperplan optimal $ \mathbf{w}^\star $** : 
  $$
  \mathbf{w}^\star = \sum_{i=1}^{n} \alpha_i^\star y_i \mathbf{x}_i
  $$
- **Biais de l'hyperplan optimal $ b^\star $** : Le biais optimal peut être ensuite calculé à partir des vecteurs de support (par exemple à partir de n'importe quel $ \mathbf{x}_i $ tel que $ \alpha_i^\star > 0 $) en utilisant l'équation :
  $$
  y_i \big( (\mathbf{w}^\star)^T \mathbf{x}_i + b^\star)\big) = 1
  $$


## Génération d'un jeu de données linéairement séparable

In [ ]:
from sklearn.datasets import make_classification

Avant toute chose, générons un jeu de données de classification binaire **linéairement séparable**. Pour ce TP, on va se restreindre à des échantillons à deux dimensions $\mathbf{x}_i \in \mathbb{R}^2$.

In [ ]:
# Génération d'un jeu de données linéairement séparable
X, y = make_classification(n_samples=100, n_features=2, n_informative=2, n_redundant=0, 
                           n_clusters_per_class=1, flip_y=0, class_sep=2.0, random_state=42)
# Ajout d'un bruit léger aux données
np.random.seed(seed=42)
noise = np.random.normal(scale=0.4, size=X.shape)
X += noise

cls = np.unique(y)
# Affichage des données
plt.figure(figsize=(8, 6))
plt.scatter(X[y==cls[0]][:, 0], X[y==cls[0]][:, 1], color='red', label='Classe %d'%cls[0], edgecolor='k')
plt.scatter(X[y==cls[1]][:, 0], X[y==cls[1]][:, 1], color='blue', label='Classe %d'%cls[1], edgecolor='k')
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.title('Jeu de données linéairement séparable')
plt.legend()
plt.grid(True)
plt.show()

Dans l'exemple précédent, la labellisation des classes a été faite de manière standard par `scikit-learn`, à savoir $y_i \in \{0,1\}$ pour un problème binaire.

In [ ]:
print("Labels des classes générées :",np.unique(y))

En revanche, pour la formulation du problème du SVM, les deux classes doivent être labellisée par $y_i \in \{ -1, +1 \}$

### 🛠️ 🚧 👷  À vous de jouer !

Relabellisez les données précédentes (le vecteur `y`) pour vous placer dans le cas attendu pour la résolution du SVM

In [ ]:
y = ??? # FIXME ⚠️
print("Labels des classes après relabellisation :",np.unique(y))

In [ ]:
cls = np.unique(y)
# Affichage des données
plt.figure(figsize=(8, 6))
plt.scatter(X[y==cls[0], 0], X[y==cls[0], 1], color='red', label='Classe %d'%cls[0], edgecolor='k')
plt.scatter(X[y==cls[1], 0], X[y==cls[1], 1], color='blue', label='Classe %d'%cls[1], edgecolor='k')
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.title('Jeu de données linéairement séparable')
plt.legend()
plt.grid(True)
plt.show()

## Résolution du SVM avec marge dure en utilisant `scipy.optimize.minimize`

Maintenant qu'un jeu de données linéairement séparable a été généré et labellisé en accord avec le modèle du SVM, nous allons passer à la résolution de son problème dual en utilisant tout d'abord la fonction `minimize` de `scipy.optimize`.

In [ ]:
from scipy.optimize import minimize

### 🛠️ 🚧 👷  À vous de jouer !

Résolvez le problème dual du SVM grâce à `minimize`, avec la démarche suivante :
1. Le problème dual $(D)$ étant un problème de **maximisation**, et la fonction `minimize` étant (comme son nom l'indique), une routine permettant de **minimiser** une fonction objective, reformulez tout d'abord le problème dual pour l'écrire comme un problème de minimisation.
2. Définissez explicitement la matrice $\mathbf{Q} \in \mathbb{R}^{n \times n}$ de terme général $ Q_{ij} = y_i y_j \mathbf{x}_i^T \mathbf{x}_j $.
3. Définissez la fonction objective à minimiser selon le format attendu par `minimize`.
4. Définissez les fonctions de contraintes du problème dual selon le format attendu par `minimize`.<br>
<u>Note</u> : la contrainte $\alpha_i \geq 0$ peut être définie soit via une contrainte d'inégalité, soit grâce à l'argument `bounds`.
5. Résolvez numériquement le problème dual grâce à `minimize`.

In [ ]:
# définition de la matrice Q de l'objective duale
Q = ??? # # FIXME ⚠️

In [ ]:
# définition de la fonction objective duale
def objective_function(x):
    ??? # FIXME ⚠️

In [ ]:
# definition des contraintes
def eq_constraint:
    ??? # FIXME ⚠️

def ineq_constraint:
    ??? # FIXME ⚠️
    
constraints = ??? # FIXME ⚠️

In [ ]:
result_sp = minimize(???) # FIXME ⚠️

# Affichage des résultats
print("Variable duale optimale :", result_sp.x)
print("Valeur optimale de la fonction objective :", result_sp.fun)
print("Succès de l'optimisation :", result_sp.success)
print("Message :", result_sp.message)

### 🛠️ 🚧 👷  À vous de jouer !

Une fois le problème dual résolu et le multiplicateur de Lagrange optimal $\boldsymbol \alpha^\star$ obtenu, déterminez :
1. Les vecteurs de support $\mathbf{x}_s$ (tels que $ \alpha_s^\star > 0 $)
2. Le vecteur normal $\mathbf{w}^\star$ de l'hyperplan optimal : $\displaystyle \mathbf{w}^\star = \sum_{i=1}^{n} \alpha_i^\star y_i \mathbf{x}_i$
3. Le biais $ b^\star$ de l'hyperplan optimal $y_i \big( (\mathbf{w}^\star)^T \mathbf{x}_i + b^\star)\big) = 1$ calculé à partir d'un vecteur de support. 
  

In [ ]:
alpha = result_sp.x

# Calcul des vecteurs de support
Xs_indices = ??? # FIXME ⚠️ indices (booléen) des vecteurs de support
ys = ??? # FIXME ⚠️ labels des vecteurs de support
Xs = ??? # FIXME ⚠️ coordonnées de vecteurs de support

In [ ]:
# Calcul des paramètres de l'hyperplan optimal
w_sp = ??? # FIXME ⚠️ vecteur normal à l'hyperplan
b_sp = ??? # FIXME ⚠️ biais de l'hyperplanred

### Affichage de l'hyperplan optimal

Pour finir, voici une fonction vous permettant d'afficher sur le scatterplot des données : 
- les vecteurs de supports `Xs` (et leur classe associée `ys`) que vous avez calculé à la question précédente.
- l'hyperplan optimal du SVM de paramètres $\mathbf{w}^\star$ et $b^\star$
- les hyperplans passant par les vecteurs de support des deux classes (d'équation ${(\mathbf{w}^\star)}^T \mathbf{x} + b^\star = \pm 1$).

In [ ]:
def plot_svm(X, y, Xs, ys, w_star, b_star):
    
    plt.figure(figsize=(10, 6))
    plt.grid(True)
    # Classe -1
    plt.scatter(X[y==-1,0], X[y==-1,1],
                color='red',label='Classe -1',marker='o',edgecolor='k')
    plt.scatter(Xs[ys==-1,0], Xs[ys==-1,1],
                color='red',label='Vecteurs de support -1',marker='o',edgecolors='k',s=150)
    # Classe +1
    plt.scatter(X[y==1,0], X[y==1,1],color='blue', label='Classe +1',marker='o',edgecolor='k')
    plt.scatter(Xs[ys==1,0], Xs[ys==1,1], 
                color='blue',label='Vecteurs de support +1',marker='o',edgecolors='k',s=150)
      
    xmin = X[:,0].min()-0.5
    xmax = X[:,0].max()+0.5
    # Hyperplan optimal
    x_vals = np.linspace(xmin, xmax, 200)
    y_vals = -(w_star[0]*x_vals+b_star)/w_star[1]
    plt.plot(x_vals, y_vals, 'k-', label='Hyperplan optimal')
    # Hyperplans passant par les vecteurs de support (w.x+b=±1)
    y_vals_support1 = -(w_star[0]*x_vals+(b_star-1))/w_star[1]
    y_vals_support2 = -(w_star[0]*x_vals+(b_star+1))/w_star[1]
    plt.plot(x_vals, y_vals_support1, 'k--', label='Hyperplan +1')
    plt.plot(x_vals, y_vals_support2, 'k-.', label='Hyperplan -1')
    # Titre, labels et légende
    plt.xlabel('$x_1$')
    plt.ylabel('$x_2$')
    plt.title('Hyperplan optimal et vecteurs de support')
    plt.legend()
    plt.show()

### 🛠️ 🚧 👷  À vous de jouer !

Affichez la solution que vous avez trouvé pour le SVM. Si celle-ci est correcte, la figure devrait faire sens...  

In [ ]:
plot_svm(X,y,Xs,ys,w_sp,b_sp)

## Résolution du SVM avec marge dure en utilisant `cvxopt`

Vous devriez être convaincus que le problème dual du SVM est bien un programme quadratique. Il est donc maintenant temps de passer à sa résolution en vous servant du solveur `qp` de `cvxopt` (qui, rappel, ne fonctionne **que** pour les programmes quadratiques au contraire de `minimize`), dont le format standard pour le solveur est 

$$\begin{aligned}
& \text{minimiser} & \frac{1}{2} \mathbf{x}^T \mathbf{P} \mathbf{x} + \mathbf{q}^T \mathbf{x} \\
& \text{sous les contraintes} & \mathbf{G}\mathbf{x} \leq \mathbf{h} \\
& & \mathbf{A}\mathbf{x} = \mathbf{b}
\end{aligned}
$$

Pour résoudre le problème dual du SVM avec `qp`, il vous faudra donc "juste" spécifier (au format `matrix` de `cvxopt`) les définitions de ces matrices $\mathbf{P}$, $\mathbf{G}$, $\mathbf{A}$ et des vecteurs $\mathbf{q}$, $\mathbf{h}$, $\mathbf{b}$.

In [ ]:
import cvxopt
from cvxopt import printing
cvxopt.matrix_repr = printing.matrix_str_default
from cvxopt import matrix
from cvxopt.solvers import qp

### 🛠️ 🚧 👷  À vous de jouer !

Résolvez le problème dual du SVM grâce au solveur `qp` de `cvxopt`, avec la démarche suivante :
1. Tout comme pour `minimize`, le solveur `qp` de `cvxopt` permet de **minimiser** une fonction objective donnée. La reformulation du problème dual que vous avez (normalement) fait à la question précédente avec `minimize` reste donc valable ici...
2. Identifiez puis définissez les matrices $\mathbf{P}$, $\mathbf{G}$, $\mathbf{A}$ et les vecteurs $\mathbf{q}$, $\mathbf{h}$, $\mathbf{b}$ attendus par `qp`.
5. Résolvez numériquement le problème dual grâce à `qp`.

⚠️ Il est possible que vous obteniez une erreur du type 
```python
TypeError: 'A' must be a 'd' matrix with 100 columns
```

`cvxopt` utilise plusieurs formats de stockage des matrices, en fonction du type des entrées (entiers, flottants, complexes). Si vous initialisez un `matrix` à partir d'un `array` entier, vous obtiendrez un encodage entier (sans surprise).

In [ ]:
A = np.array([2,1])
matrix(A).typecode

L'erreur précédente vous indique que `qp` attend que le paramètre (matrice ou vecteur) qui lui est passés en arguments soit de type `'d'` (flottant). Pour cela, il suffit juste de multiplier l'`array` qui sert à instancier la variable `matrix` par `1.`

In [ ]:
A = np.array([2,1])
matrix(1.*A).typecode

In [ ]:
# définition des matrices/vecteurs (au format matrix) pour le problème dual
P = ??? # FIXME ⚠️
q = ??? # FIXME ⚠️
G = ??? # FIXME ⚠️
h = ??? # FIXME ⚠️
A = ??? # FIXME ⚠️
b = ??? # FIXME ⚠️

In [ ]:
# résolution du problème dual
result_cvxopt = qp(???) # FIXME ⚠️

### 🛠️ 🚧 👷  À vous de jouer !

Idem que pour la question précédente avec `minimize`, déterminez maintenant grâce au multiplicateur de Lagrange optimal $\boldsymbol \alpha^\star$ obtenu avec `qp` :
1. Les vecteurs de support $\mathbf{x}_s$ (tels que $ \alpha_s^\star > 0 $)
2. Le vecteur normal $\mathbf{w}^\star$ de l'hyperplan optimal : $\displaystyle \mathbf{w}^\star = \sum_{i=1}^{n} \alpha_i^\star y_i \mathbf{x}_i$
3. Le biais $ b^\star$ de l'hyperplan optimal $y_i \big( (\mathbf{w}^\star)^T \mathbf{x}_i + b^\star)\big) = 1$ calculé à partir d'un vecteur de support. 
  

In [ ]:
alpha = np.array(result_cvxopt['x']).flatten()

# Calcul des vecteurs de support
Xs_indices = ??? # FIXME ⚠️ indices (booléen) des vecteurs de support
ys = ??? # FIXME ⚠️ labels des vecteurs de support
Xs = ??? # FIXME ⚠️ coordonnées de vecteurs de support

In [ ]:
# Calcul des paramètres de l'hyperplan optimal
w_cvxopt = ??? # FIXME ⚠️ vecteur normal à l'hyperplan
b_cvxopt = ??? # FIXME ⚠️ biais de l'hyperplan

### 🛠️ 🚧 👷  À vous de jouer !

Affichez la solution que vous avez trouvé pour le SVM avec `qp`. Si celle-ci est correcte (et sous réserve que celle que vous avez obtenu avec `minimize` l'était aussi), vous devriez donc obtenir la même chose...

In [ ]:
plot_svm(X,y,Xs,ys,w_cvxopt,b_cvxopt)

### 🛠️ 🚧 👷  À vous de jouer !

Sous réserve que vous avez trouvé les bonnes solutions avec `minimize` et `qp`, l'affichage du scatterplot avec les vecteurs de support et hyperplans optimaux doit normalement être le même dans les deux cas de figure (ouf).

Vérifiez donc que les paramètres $\mathbf{w}^\star$ et $b^\star$ concordent bien numériquement pour les deux méthodes : 
- est-ce vraiment le cas ?
- que pouvez vous en conclure ?

In [ ]:
w_sp ??? w_cvxopt # FIXME ⚠️ est-ce que les vecteurs normaux aux hyperplans concordent ?
b_sp ??? b_cvxopt # FIXME ⚠️ est-ce que les biais des hyperplans concodent ?

On peut donc en conclude que ⚠️ FIXME

## Bravo ! 👏🍻

Vous en avez terminé avec la résolution du SVM à marge dure. En route maintenant vers les [SVMs à marge douce](TP_SVM_exo3.ipynb) !